<table>
    <tr>
        <td><img src="./img/Macc.png" width="400"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="color:blue;text-align:left">Álgebra Lineal</h1></td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Sesión 8</p></tp>
            <tp><p style="font-size:150%;text-align:center">Factorización LU</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

## Objetivos

* Comprender la idea general de la factorización LU.
* Implementar los algoritmos principales de la factorización LU y las sustituciones hacia atrás y hacia adelante.

Adaptado de Strang (2009), capítulo 2, sección 2.6.

## Integrando todos los conceptos hasta ahora

Es momento de juntar todo lo que hemos construido hasta ahora e integrarlo en una solución computacional para resolver sistemas de ecuaciones lineales.

Recordemos que hemos visto cómo construir matrices de eliminación $E_{i_1},\ldots, E_{i_s}$ de tal manera que $E_{i_1}\ldots E_{i_s}A$ sea una matriz triangular superior $U$. Esto es, la eliminación consiste en transformar el sistema matricial

$$
A\pmb{x}=\pmb{b}
$$

en el sistema

$$
E_{i_1}\ldots E_{i_s}A\pmb{x}=E_{i_1}\ldots E_{i_s}\pmb{b}
$$

Observe que $E_{i_1}\ldots E_{i_s}A=U$ (una matriz triangular superior) y llamemos a $E_{i_1}\ldots E_{i_s}\pmb{b}$ como $\pmb{c}$. Así pues, el sistema $U\pmb{x}=\pmb{b}$ puede usarse para encontrar los valores de las incógnitas mediante sustitución hacia atrás.

**Nota:** Esto se cumple para todas las matrices $A$ cuyo método de eliminación no requiera cambios de filas para disponer de pivotes. Más adelante volveremos sobre el caso general.

**Ejemplo:**

Por ejemplo, considere el siguiente sistema de ecuaciones:

$$
\begin{align}
x + 2y &= 5\\
4x + 9y &= 21
\end{align}
$$

Su forma matricial es

$$
\begin{bmatrix}
1 & 2\\
4 & 9
\end{bmatrix}\begin{bmatrix}
x\\
y
\end{bmatrix} = \begin{bmatrix}
5\\
21
\end{bmatrix}
$$

El método de eliminación gaussiana consiste en usar los pivotes de $A$ para eliminar los coeficientes de las filas inferiores, de tal manera que se obtenga una matriz triangular superior $U$. En nuestro ejemplo, tenemos $E_{21}A=U$

$$
\begin{bmatrix}
1 & 0\\
-4 & 1
\end{bmatrix}\begin{bmatrix}
1 & 2\\
4 & 9
\end{bmatrix} = \begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}
$$

donde $E_{21}=\begin{bmatrix}
1 & 0\\
-4 & 1
\end{bmatrix}$ y $U=\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}$.

En el lado derecho de la ecuación funciona igual, en donde obtenemos $E_{21}\pmb{b}=\pmb{c}$

$$
\begin{bmatrix}
1 & 0\\
-4 & 1
\end{bmatrix}\begin{bmatrix}
5\\
21
\end{bmatrix} = \begin{bmatrix}
5\\
1
\end{bmatrix}
$$


## Sustitución hacia atrás


Comenzaremos primero automatizando el final, es decir, una vez transformada la matriz original $A$ en una matriz triangular superior $U$, procedemos a hacer la sustitución hacia atrás para encontrar el valor de cada incógnita. Veámos esto en nuestro ejemplo.

Tenemos 

$$\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}\begin{bmatrix}
x\\
y
\end{bmatrix} = \begin{bmatrix}
5\\
1
\end{bmatrix}
$$

de donde obtenemos $y=1$ y 

$$
x = \frac{\stackrel{\stackrel{\mbox{primer elemento de $\pmb{c}$}}{\downarrow}}{5} - \stackrel{\stackrel{\mbox{elemento $(1,2)$ de $U$}}{\downarrow}}{2}*\stackrel{\stackrel{\mbox{$y$}}{\downarrow}}{1}}{\stackrel{1}{\stackrel{\uparrow}{\mbox{primer pivote}}}}=3
$$

El siguiente código permite encontrar las soluciones dado un sistema triangular $U\pmb{x}=\pmb{c}$:

In [1]:
import numpy as np

def sustitucion_atras(U, c) :
    n, n = U.shape
    solucion = [np.nan] * n # Inicializamos el vector solución
    for k in range(n-1, -1, -1) : # Para cada k desde n hasta 1
        ck = c.item(k) # valor del elemento k-esimo del vector c
        t = 0 # Inicializamos la suma de elementos U(k,j) multiplicados por las soluciones j-ésimas
        for j in range(k + 1, n) : # para cada j desde k+1 hasta n
            t += U.item((k,j)) * solucion[j] # valor del elemento U(k,j) por solución j-ésima
        solucion[k] = (ck - t)/U.item(k,k) # solución k-ésima es despeje de fila k-ésima
        print(f"{solucion[k]} = solucion[{k+1}] = (c{k+1} - t)/U({k+1},{k+1}) = ({ck} - {t}) / {U.item(k,k)}")
    return np.matrix([solucion]).T

Verificamos con nuestro ejemplo:

In [2]:
U = np.matrix([[1,2], [0,1]])
c = np.matrix([[5], [1]])
sustitucion_atras(U, c)

1.0 = solucion[2] = (c2 - t)/U(2,2) = (1 - 0) / 1
3.0 = solucion[1] = (c1 - t)/U(1,1) = (5 - 2.0) / 1


matrix([[3.],
        [1.]])

## Propiedades de las matrices triangulares inferiores

**Multiplicación**

Veámos ahora un par de propiedades importantes de las matrices triangulares inferiores. Suponga que $A$ y $B$ son las siguientes matrices:

$$
A=\begin{bmatrix}
1 & 0 & 0 & 0 \\
3 & 2 & 0 & 0 \\
2 & -1 & 3 & 0 \\
1 & 3 & 3 & 4 \\
\end{bmatrix}\qquad B = \begin{bmatrix}
1 & 0 & 0 & 0 \\
2 & 1 & 0 & 0 \\
1 & 2 & 1 & 0 \\
2 & 3 & 2 & 4 \\
\end{bmatrix}
$$

Observe que $AB$ también es triangular inferior (ver ejercicio 3 del notebook 6):

$$
AB=\begin{bmatrix}
1 & 0 & 0 & 0 \\
7 & 2 & 0 & 0 \\
3 & 5 & 3 & 0 \\
18 & 21 & 11 & 16 \\
\end{bmatrix}
$$

Esto ocurre con todas las matrices triangulares inferiores: su multiplicación también es una matriz triangular inferior.

**Inversa**

Lo mismo ocurre con las inversas de matrices triangulares inferiores. Suponga que $A$ es la siguiente matriz:

$$
A=\begin{bmatrix}
1 & 0 & 0\\
3 & 1 & 0\\
4 & 5 & 1
\end{bmatrix}
$$

Entonces su inversa es (ver ejercicio 2 del notebook 7):

$$
A^{-1}=\begin{bmatrix}
1 & 0 & 0\\
-3 & 1 & 0\\
11 & -5 & 1
\end{bmatrix}
$$

Observe que $A^{-1}$ también es triangular inferior. Esto ocurre con todas las matrices triangulares inferiores: su inversa también es una matriz triangular inferior.

## Factorización LU

Observe que para resolver el sistema $A\pmb{x}=\pmb{b}$ usamos matrices de eliminación para que los pivotes eliminen a los coeficientes debajo de cada uno de ellos. Estas matrices de eliminación son matrices triangulares inferiores $E_{i_1},\ldots, E_{i_s}$. Así pues, por las propiedades que acabamos de ver, su multiplicación también es una matriz triangular inferior, y su inversa también lo es. Esto es, $(E_{i_1}\ldots E_{i_s})^{-1}$ es una matriz triangular inferior. Como teníamos que

$$
E_{i_1}\ldots E_{i_s}A=U
$$

entonces tenemos que

$$A=(E_{i_1}\ldots E_{i_s})^{-1}U$$

Si llamamos $(E_{i_1}\ldots E_{i_s})^{-1}=L$, entonces obtenemos la factorización $LU$ de $A$

$$A=LU$$

donde $L$ es una matriz triangular inferior y $U$ es una matriz triangular superior.


**Nota:** Esto se cumple para todas las matrices $A$ cuyo método de eliminación no requiera cambios de filas para disponer de pivotes. Más adelante volveremos sobre el caso general.

**Ejemplo:**

Por ejemplo, si $A=\begin{bmatrix}
1 & 2\\
4 & 9
\end{bmatrix}$, entonces

$$
A = \begin{bmatrix}
1 & 0\\
4 & 1
\end{bmatrix}\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix} 
$$

donde 

$$
L=E_{21}^{-1}=\left(\begin{bmatrix}
1 & 0\\
-4 & 1
\end{bmatrix}\right) = \begin{bmatrix}
1 & 0\\
4 & 1
\end{bmatrix}
$$


**Ejemplo:**

Para encontrar la factorización $LU$ de $A$, donde

$$
A=\begin{bmatrix}
2 & 3\\
4 & 7
\end{bmatrix}
$$

Necesitamos usar el primer pivote 2 para eliminar el coeficiente debajo de él, 4. Para ello usamos la matriz

$$
E_{21}=\begin{bmatrix}
1 & 0\\
-2 & 1
\end{bmatrix}
$$

In [3]:
A = np.matrix([[2,3], [4,7]])
E = np.matrix([[1,0], [-2, 1]])
print("A=\n", A)
print("E=\n", E)
print("EA=\n", E*A)

A=
 [[2 3]
 [4 7]]
E=
 [[ 1  0]
 [-2  1]]
EA=
 [[2 3]
 [0 1]]


Encontramos que 

$$
U=\begin{bmatrix}
2 & 3\\
0 & 1
\end{bmatrix}
$$

y que 

$$
L=E_{21}^{-1}=\left(\begin{bmatrix}
1 & 0\\
-2 & 1
\end{bmatrix}\right) = \begin{bmatrix}
1 & 0\\
2 & 1
\end{bmatrix}
$$

Luego 

$$
A = LU = \begin{bmatrix}
1 & 0\\
2 & 1
\end{bmatrix}\begin{bmatrix}
2 & 3\\
0 & 1
\end{bmatrix}
$$

Verificamos mediante numpy:

In [4]:
L = np.matrix([[1,0], [2, 1]])
U = np.matrix([[2,3], [0, 1]])
print("L=\n", L)
print("U=\n", U)
print("LU=\n", L*U)
print("LU=A?\n", L*U==A)

L=
 [[1 0]
 [2 1]]
U=
 [[2 3]
 [0 1]]
LU=
 [[2 3]
 [4 7]]
LU=A?
 [[ True  True]
 [ True  True]]


**Ejemplo:**

Encuentre la factorización $LU$ de

$$
A=\begin{bmatrix}
1 & 1 & 1 & 1\\
1 & 2 & 3 & 4\\
1 & 3 & 6 & 10\\
1 & 4 & 10 & 20\\
\end{bmatrix}
$$

Dado que la primera columna es solo 1s, entonces las matrices de eliminación son:

$$
E_{21}=\begin{bmatrix}
1 & 0 & 0 & 0\\
-1 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}\qquad E_{31}=\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
-1 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}\qquad E_{41}=\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
-1 & 0 & 0 & 1\\
\end{bmatrix} 
$$

Observe cómo eliminamos los coeficientes debajo de la primera columna:

In [5]:
A = np.matrix([[1,1,1,1], [1,2,3,4], [1,3,6,10], [1,4,10,20]])
E21 = np.matrix([[1,0,0,0], [-1,1,0,0], [0,0,1,0], [0,0,0,1]])
E31 = np.matrix([[1,0,0,0], [0,1,0,0], [-1,0,1,0], [0,0,0,1]])
E41 = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [-1,0,0,1]])
E41*E31*E21*A

matrix([[ 1,  1,  1,  1],
        [ 0,  1,  2,  3],
        [ 0,  2,  5,  9],
        [ 0,  3,  9, 19]])

Observe que es muy sencillo encontrar las inversas de $E_{21}$, $E_{31}$ y $E_{41}$:

$$
E_{21}^{-1}=\begin{bmatrix}
1 & 0 & 0 & 0\\
1 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}\qquad E_{31}^{-1}=\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
1 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}\qquad E_{41}^{-1}=\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
1 & 0 & 0 & 1\\
\end{bmatrix} 
$$

Con lo cual se puede obtener una matriz triangular inferior:

In [6]:
E21inv = np.matrix([[1,0,0,0], [1,1,0,0], [0,0,1,0], [0,0,0,1]])
E31inv = np.matrix([[1,0,0,0], [0,1,0,0], [1,0,1,0], [0,0,0,1]])
E41inv = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [1,0,0,1]])
E21inv*E31inv*E41inv

matrix([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 0, 1, 0],
        [1, 0, 0, 1]])

Esta todavía no es la $L$ que buscamos, pues no hemos terminado el proceso de eliminación.

**Ejercicio 1:**

Complete el proceso de factorización $LU$ de $A$.

**Nota:**
Observe que $(E_{i_1}\ldots E_{i_s})^{-1}=E_{i_s}^{-1}\ldots E_{i_1}^{-1}$. Esto simplifica encontrar $L$.

**Respuesta:**

Encontramos las matrices de eliminación de los coeficientes debajo del segundo pivote:

In [7]:
E32 = np.matrix([[1,0,0,0], [0,1,0,0], [0,-2,1,0], [0,0,0,1]])
E42 = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,-3,0,1]])
E42*E32*E41*E31*E21*A

matrix([[ 1,  1,  1,  1],
        [ 0,  1,  2,  3],
        [ 0,  0,  1,  3],
        [ 0,  0,  3, 10]])

Encontramos la matriz para eliminar el coeficiente debajo del tercer pivote:

In [8]:
E43 = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,-3,1]])
U = E43*E42*E32*E41*E31*E21*A
print("U=\n", U)

U=
 [[1 1 1 1]
 [0 1 2 3]
 [0 0 1 3]
 [0 0 0 1]]


Encontramos las inversas de todas las matrices de eliminación:

In [9]:
E21inv = np.matrix([[1,0,0,0], [1,1,0,0], [0,0,1,0], [0,0,0,1]])
E31inv = np.matrix([[1,0,0,0], [0,1,0,0], [1,0,1,0], [0,0,0,1]])
E41inv = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [1,0,0,1]])
E32inv = np.matrix([[1,0,0,0], [0,1,0,0], [0,2,1,0], [0,0,0,1]])
E42inv = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,3,0,1]])
E43inv = np.matrix([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,3,1]])
L = E21inv*E31inv*E41inv*E32inv*E42inv*E43inv
print("L=\n", L)

L=
 [[1 0 0 0]
 [1 1 0 0]
 [1 2 1 0]
 [1 3 3 1]]


In [10]:
L*U

matrix([[ 1,  1,  1,  1],
        [ 1,  2,  3,  4],
        [ 1,  3,  6, 10],
        [ 1,  4, 10, 20]])

In [11]:
A==L*U

matrix([[ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])

---

**Ejercicio 2$^*$:**

Implemente un código para realizar de manera automática la factorización $LU$ de una matriz. Puede usar el siguiente pseudo-código para inspirarse:

<img src="./img/pseudo-codigo.png" width="auto"/>

## Resolver el sistema

Observe que hasta ahora hemos buscado la factorización $LU$ de una matriz $A$. Todavía no hemos precisado cómo usar dicha factorización para resolver un sistema $A\pmb{x}=\pmb{b}$. En efecto, la factorización sólo habla de $A$, mas no involucra para nada a $\pmb{b}$. Más que un defecto del procedimiento, esto es una virtud. Se determina cómo solucionar cualquier sistema independientemente de $\pmb{b}$. Pero, ¿cómo involucramos a $\pmb{b}$?

Como mencionamos al comienzo, hemos transformado el sistema matricial

$$
A\pmb{x}=\pmb{b}
$$

en el sistema

$$
L^{-1}A\pmb{x}=L^{-1}\pmb{b}
$$

Esto es,

$$
U\pmb{x}=\pmb{c}
$$

donde $U=L^{-1}A$ y $\pmb{c}=L^{-1}\pmb{b}$.

Ahora bien, observe que $L\pmb{c}=\pmb{b}$ y que $L$ es una matriz triangular inferior. Entonces podemos usar sustitución hacia adelante para obtener $\pmb{c}$ y luego sustitución hacia atrás con $U\pmb{x}=\pmb{c}$ para obtener $\pmb{x}$.

**Ejemplo:**

Consideremos de nuevo el sistema

$$
\begin{bmatrix}
1 & 2\\
4 & 9
\end{bmatrix}\begin{bmatrix}
x_1\\
x_2
\end{bmatrix} = \begin{bmatrix}
5\\
21
\end{bmatrix}
$$

La factorización $LU$ permite encontrar que

$$
A = \begin{bmatrix}
1 & 0\\
4 & 1
\end{bmatrix}\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}
$$

donde $L=\begin{bmatrix}
1 & 0\\
4 & 1
\end{bmatrix}$ y $U=\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}$.

Resolvemos mediante sustitución hacia adelante el sistema $L\pmb{c}=\pmb{b}$:

$$
\begin{bmatrix}
1 & 0\\
4 & 1
\end{bmatrix}\begin{bmatrix}
c_1\\
c_2
\end{bmatrix} = \begin{bmatrix}
5\\
21
\end{bmatrix}
$$

Se tiene que $c_1=5$ y que

$$
c_2 = \frac{21 - 4*5}{1} = 1
$$

Así, $\pmb{c}=\begin{bmatrix}
5\\
1
\end{bmatrix}$.

Ahora resolvemos mediante sustitución hacia atrás el sistema $U\pmb{x}=\pmb{c}$:

$$
\begin{bmatrix}
1 & 2\\
0 & 1
\end{bmatrix}\begin{bmatrix}
x_1\\
x_2
\end{bmatrix} = \begin{bmatrix}
5\\
1
\end{bmatrix}
$$

Se tiene que $x_2=1$ y que

$$
x_1 = \frac{5 - 2*1}{1} = 3
$$

Así, $\pmb{x}=\begin{bmatrix}
3\\
1
\end{bmatrix}$.

**Ejercicio 3$^*$:**

Implemente un código para realizar de manera automática la sustitución hacia adelante.

**Respuesta:**

Una posible implementación es la siguiente:

In [12]:
def sustitucion_adelante(L, b) :
    n, n = L.shape
    solucion = [np.nan] * n # Inicializamos el vector solución
    for k in range(n) : # Para cada k desde n hasta 1
        bk = b.item(k) # valor del elemento k-esimo del vector b
        s = 0 # Inicializamos la suma de elementos U(k,j) multiplicados por las soluciones j-ésimas
        for j in range(k) : # para cada j desde 1 hasta k-1
            s += L.item((k,j)) * solucion[j] # valor del elemento U(k,j) por solución j-ésima
        solucion[k] = (bk - s)/L.item(k,k) # solución k-ésima es despeje de fila k-ésima
        print(f"{solucion[k]} = solucion[{k+1}] = (b{k+1} - s)/L({k+1},{k+1}) = ({bk} - {s}) / {L.item(k,k)}")
    return np.matrix([solucion]).T

In [13]:
L = np.matrix([[1,0], [4,1]])
U = np.matrix([[1,2], [0,1]])
b = np.matrix([[5], [21]])
print("Solucionando sistema Lc=b...")
c = sustitucion_adelante(L, b)
print("Solucionando sistema Ux=c...")
sustitucion_atras(U, c)

Solucionando sistema Lc=b...
5.0 = solucion[1] = (b1 - s)/L(1,1) = (5 - 0) / 1
1.0 = solucion[2] = (b2 - s)/L(2,2) = (21 - 20.0) / 1
Solucionando sistema Ux=c...
1.0 = solucion[2] = (c2 - t)/U(2,2) = (1.0 - 0) / 1
3.0 = solucion[1] = (c1 - t)/U(1,1) = (5.0 - 2.0) / 1


matrix([[3.],
        [1.]])

## Ejercicios

**Ejercicio 4:**

Encuentre la factorización $LU$ de las siguientes matrices:

$$
A = \begin{bmatrix}
2 & 1 & 0\\
0 & 4 & 2\\
6 & 3 & 5\\
\end{bmatrix}\qquad A = \begin{bmatrix}
1 & 1 & 1\\
2 & 4 & 5\\
6 & 4 & 0\\
\end{bmatrix}
$$

**Ejercicio 5:**
    
¿Qué matriz diagonal $D$ permite que la matriz $DU$ tenga solo 1s en la diagonal, donde

$$
U = \begin{bmatrix}
2 & 4 & 8\\
0 & 3 & 9\\
0 & 0 & 7\\
\end{bmatrix}
$$

**Ejercicio 6:**
    
Suponga que 

$$
A = \begin{bmatrix}
2 & 4\\
4 & 11\\
\end{bmatrix}
$$

Encuentre una factorización $LDU$ de $A$ de tal manera que $L$ sea una matriz triangular superior, $D$ una matriz diagonal y $U$ solo tenga 1s en la diagonal.

**Ejercicio 7:**

Resuelva los sistemas $L\pmb{c}=\pmb{b}$ y $U\pmb{x}=\pmb{c}$ para

$$
L = \begin{bmatrix}
1 & 0 & 0\\
1 & 1 & 0\\
1 & 1 & 1
\end{bmatrix}\qquad U = \begin{bmatrix}
1 & 1 & 1\\
0 & 1 & 1\\
0 & 0 & 1\\
\end{bmatrix}\qquad \pmb{b} = \begin{bmatrix}
4\\
5\\
6\\
\end{bmatrix}
$$

## Revisión de las ideas principales

* La eliminación gaussiana (sin intercambio de filas) factoriza $A$ en $LU$.

* La matriz triangular inferior $L$ contiene los multiplicadores $\ell_{ij}$ que multiplican las filas pivote, para ir de $A$ a $U$. El producto $LU$ restituye esas filas para recuperar $A$.

* Para incluir el lado derecho resolvemos $L\pmb{c}=\pmb{b}$ y $U\pmb{x}=\pmb{c}$.


## Referencias

- Strang, Gilbert (2009) *Introduction to Linear Algebra*, 4th Edition. Wellesley-Cambridge Press, 2009.
